In [8]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import sys

In [13]:
df = pd.read_csv(
    '/content/train.csv',
    encoding='latin1',
    on_bad_lines='skip',
    engine='python'
)


In [16]:
text = " ".join(df['title'].dropna().astype(str)).lower()
vocab = sorted(set(text))
print(f'Vocabulary size: {len(vocab)}')

char2idx = {c: i for i, c in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Vocabulary size: 61


In [17]:
seq_length = 100

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [18]:
vocab_size = len(vocab)
embedding_dim = 64
rnn_units = 128

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape=(None,)),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])


def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 64)       │         3,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 128)      │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 61)       │         7,869 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,589 (431.99 KB)

 Trainable params: 110,589 (431.99 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 35s 315ms/step - loss: 3.3270
Epoch 2/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 23s 216ms/step - loss: 2.5845
Epoch 3/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 42s 225ms/step - loss: 2.3852
Epoch 4/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 245ms/step - loss: 2.2700
Epoch 5/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 24s 225ms/step - loss: 2.1919
Epoch 6/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 40s 221ms/step - loss: 2.1311
Epoch 7/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 22s 214ms/step - loss: 2.0764
Epoch 8/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 48s 283ms/step - loss: 2.0318
Epoch 9/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 28s 266ms/step - loss: 1.9889
Epoch 10/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 41s 270ms/step - loss: 1.9484
Epoch 11/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 240ms/step - loss: 1.9158
Epoch 12/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 24s 226ms/step - loss: 1.8851
Epoch 13/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 40s 226ms/step - loss: 1.8528
Epoch 14/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 24s 233ms/step - loss: 1.8223
Epoch 15/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 

In [20]:
def generate_text(model, start_string, num_generate=100, temperature=1.0):
    input_eval = [char2idx.get(s, 0) for s in start_string.lower()]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.layers[1].reset_states()

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(
            predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)


print(generate_text(model, start_string="The ", num_generate=200, temperature=0.8))

The ewama orunorke -&nimpan, s culliseur uprakicadiacad mpte $7' st d oro rumpe ps mbaloumamas seracldisund r tiasin tn, e dns tewstous." rutit fotharexin s s turs hem ifo uplllan tican man t h inacawny p
